### Step 1: Import packages

In [21]:
import pandas as pd
import numpy as np
import netCDF4 as ncdf
import os
from datetime import date, timedelta
from math import pi
import fiona

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

# geography
import geopandas as gpd
import osmnx as ox
import shapely
from shapely.geometry import Point
import contextily as ctx 

#Moved from sklearn.neighbors to sklearn.metrics following their package change
import sklearn.metrics
dist = sklearn.metrics.DistanceMetric.get_metric(
    'haversine'
)

# ignore warnings
import warnings
warnings.filterwarnings(
    'ignore'
)


### Step 2: Define working directories

In [2]:
#Local directories on my machine (not gdrive)
in_dir_zip_shapes = '../data/raw/wind/tl_2010_06_zcta510/'
in_dir = '../data/processed/'
in_health = '../data/raw/health/'
out_dir = '../data/processed/'

In [22]:
# #Local directories on windows machine (not gdrive)
# in_dir_zip_shapes = 'tl_2010_06_zcta510\\'
# in_dir = 'wind\\ca only\\'
# in_health = 'health\\'
# out_dir = 'wind\\clean\\'

### Step 3: Read data

``wind``

In [4]:
df = pd.read_csv(in_dir + "all_years_wind_data.csv",index_col=0)

df.head()

,Unnamed: 0,lat,lon,ZCTA10,u,v,wdir,wspd,year_month,year,month
0,0,37.465,-117.936,89010,0.992558,0.124684,7.159901,1.000358,200601,2006,1
1,1,35.396,-116.322,89019,0.088867,-0.147450,301.077087,0.172160,200601,2006,1
2,2,36.161,-116.139,89060,-0.319009,-0.046086,188.220367,0.322321,200601,2006,1
3,3,35.957,-115.897,89061,-0.106072,-0.459862,257.011322,0.471937,200601,2006,1
4,4,39.520,-120.032,89439,0.707161,0.798291,48.464073,1.066463,200601,2006,1


In [13]:
test_df = gdf[['lat','lon','u','v','wdir','wspd','geometry']].head(10)

In [14]:
test_df

,lat,lon,u,v,wdir,wspd,geometry
0,37.465,-117.936,0.992558,0.124684,7.159901,1.000358,POINT (-13128575.466 4504121.513)
1,35.396,-116.322,0.088867,-0.147450,301.077087,0.172160,POINT (-12948905.808 4217827.033)
2,36.161,-116.139,-0.319009,-0.046086,188.220367,0.322321,POINT (-12928534.341 4322797.398)
3,35.957,-115.897,-0.106072,-0.459862,257.011322,0.471937,POINT (-12901595.024 4294706.250)
4,39.520,-120.032,0.707161,0.798291,48.464073,1.066463,POINT (-13361901.119 4796433.202)
5,33.959,-118.247,-0.667049,-1.111095,239.021393,1.295950,POINT (-13163195.828 4023298.056)
6,33.937,-118.229,-0.667049,-1.111095,239.021393,1.295950,POINT (-13161192.077 4020345.799)
7,33.938,-118.281,-0.667049,-1.111095,239.021393,1.295950,POINT (-13166980.691 4020479.976)
8,34.069,-118.338,-0.667049,-1.111095,239.021393,1.295950,POINT (-13173325.901 4038070.808)
9,34.055,-118.328,-0.667049,-1.111095,239.021393,1.295950,POINT (-13172212.707 4036189.575)


#### Haversine

In [12]:
# Radius of earth in km
r = 6378.100

In [31]:
def hav_dist(a,b):
    """
    Calculate the Haversine distance between two points of lat lon
    """
    
    pa_lat = a['lat']
    pa_lon = a['lon']
    pb_lat = b['lat']
    pb_lon = b['lon']

    s_lat = np.sin((np.deg2rad(pb_lat - pa_lat))/2)**2 
    s_lon = np.sin((np.deg2rad(pb_lon - pa_lon))/2)**2
    c_lat = np.cos(np.deg2rad(pb_lat)) * np.cos(np.deg2rad(pa_lat)) 
    tot = s_lat+s_lon*c_lat
    dist = 2*r*np.arcsin(np.sqrt(tot)) * 0.621371
    
    return dist

hav_dist(test_df.iloc[4,:],test_df.iloc[3,:]) 

334.4335870463861

In [42]:
def bearing(a,b):
    """
    Calculate the bearing from point b to point a
    """
    
    pa_lat = a['lat']
    pa_lon = a['lon']
    pb_lat = b['lat']
    pb_lon = b['lon']
    
    if pb_lon > pa_lon:
        d_X = -np.cos(pa_lat) * np.sin(pa_lon - pb_lon)
    else:
        d_X = np.cos(pa_lat) * np.sin(pa_lon - pb_lon)

    if pb_lat >pb_lon:
        d_Y = -np.cos(pb_lat) * np.sin(pa_lat) - np.sin(pb_lat) * np.cos(pa_lat) * np.cos(pa_lon - pb_lon)
    else:
        d_Y = np.cos(pb_lat) * np.sin(pa_lat) - np.sin(pb_lat) * np.cos(pa_lat) * np.cos(pa_lon - pb_lon)

    B_ba = np.arctan2(d_X,d_Y)*(180/np.pi)%360
    return B_ba
    
bearing(test_df.iloc[3,:],test_df.iloc[4,:])

156.8467318755927

In [44]:
def wind_direction(u,v):
    """
    Calculate direction of wind in degrees as bearing angle from true North
    """
    
    wind_dir =  np.arctan2(u,v)*(180/np.pi)%360
    return wind_dir

wind_direction(test_df.iloc[4,:]['u'],test_df.iloc[4,:]['v'])

41.53592367546234

In [47]:
def interaction(bearing, wdir_ba):
    """
    Calculate interaction term of wind and bearing based on difference of angles.
    """
    
    angle_diff = np.max([bearing,wdir_ba]) - np.min([bearing,wdir_ba])
    if angle_diff >180:
        upwind_effect = (angle_diff)/360
    else:
        upwind_effect = 1-(angle_diff)/360
    return upwind_effect

interaction(bearing(test_df.iloc[3,:],test_df.iloc[4,:]), wind_direction(test_df.iloc[4,:]['u'],test_df.iloc[4,:]['v']))

0.6796921994440823